<a href="https://colab.research.google.com/github/fco-dv/ignite-cut/blob/main/cut.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Contrastive Learning for UnpairedImage-to-Image Translation (CUT) with Ignite and `torch.cuda.amp`

In this notebook we provide an implementation of [CUT](https://arxiv.org/pdf/2007.15651) and its training on "Horse 2 Zebra" dataset using Ignite.

In contrast, we will use recently added [`torch.cuda.amp`](https://pytorch.org/docs/master/notes/amp_examples.html#working-with-multiple-models-losses-and-optimizers) module to perform automatic mixed precision training instead of using Nvidia/Apex package. This module is available in pytorch (>=1.6.0) release.


### CUT in a Nutshell

CUT only requires learning the mapping in one direction and avoids using inverse auxiliary generators and discriminators. This can largely simplify the training procedure and reduce training time. The generator function $G$ is broken up into two components, an encoder $Genc$ followed by a decoder $Gdec$, which are applied sequentially to produce output image.

$$
\hat{y} = G(x) = Gdec(Genc(x))
$$


and two discriminators $D_A$ and $D_B$. Training of the networks is done by minimizing the loss is a sum of 3 components:
$$
\mathcal{L}(G, F, D_A, D_B) = \mathcal{L}_{GAN}(G, D_B) + \mathcal{L}_{GAN}(F, D_A) + \lambda \mathcal{L}_{cyc}(G, F)
$$
with GAN loss:
$$
\mathcal{L}_{GAN}(G, D_B) = \text{mean}_{x \in A}\left[ (D_B(G(x)) - 1)^2 \right]+ \text{mean}_{y \in B}\left[ (D_B(y) - 1)^2 \right] \\
\mathcal{L}_{GAN}(F, D_A) = \text{mean}_{y \in B}\left[ (D_A(F(y)) - 1)^2 \right]+ \text{mean}_{x \in A}\left[ (D_A(x) - 1)^2 \right]
$$
and forward and backward cycle consistency term:
$$
\mathcal{L}_{cyc}(G, F) = \text{mean}_{x \in A}\left[ |F(G(x)) - x|_1 \right] + \text{mean}_{y \in B}\left[ |G(F(y)) - y|_1 \right]
$$

Optionally, one can add identity loss terms. See [here](https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix/blob/master/docs/qa.md#what-is-the-identity-loss-322-373-362).

SyntaxError: ignored